In [1]:
import os
import sys
import xml.etree.ElementTree as ElementTree
import collections
from collections import Counter

import psycopg2


In [9]:
connection = psycopg2.connect(dbname='loi', user='loi', password='baba')
curseur = connection.cursor()

## Parse les textes (struct et version)

In [ ]:
curseur.execute("select base_origine, categorie, cid from texte where valide = True;")
liste_texte = curseur.fetchall()


In [67]:
liste_id_par_cid = {}
valeurs_struct_par_id_par_cid = {}
valeurs_vecteur_par_id_par_cid = {}
i = 0
for base_origine, categorie, cid in liste_texte:
    liste_id_, valeurs_struct_par_id_, valeurs_version_par_id_ = parse_cid(cid)
    
    liste_id_par_cid[cid] = liste_id_
    valeurs_struct_par_id_par_cid[cid] = valeurs_struct_par_id_
    valeurs_vecteur_par_id_par_cid[cid] = valeurs_version_par_id_
    
    i += 1
    if i%1000 == 0:
        print(i)
    

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000


KeyboardInterrupt: 

## Fusionne et vérifie les informations communes au cid

In [48]:
def verifie_infos_communes(cid, liste_id_, valeurs_struct_par_id_, valeurs_version_par_id_):
    liste_cid = ([valeurs_struct_par_id_[id_]['CID'] for id_ in liste_id_] + 
                 [valeurs_version_par_id_[id_]['CID'] for id_ in liste_id_]) 
    assert (len(set(liste_cid)) == 1) and (liste_cid[0] == cid), ('{} {} {} {}'.format(cid, 'CID', set(liste_cid)))
    
    infos_communes_texte = {}
    for info_commune in infos_communes + infos_communes_struct + infos_communes_version:
        if info_commune in infos_communes:
            valeurs = ([valeurs_struct_par_id_[id_][info_commune] for id_ in liste_id_] + 
                       [valeurs_version_par_id_[id_][info_commune] for id_ in liste_id_])
        elif info_commune in infos_communes_struct:
            valeurs = [valeurs_struct_par_id_[id_][info_commune] for id_ in liste_id_]
        elif info_commune in infos_communes_version:
            valeurs = [valeurs_version_par_id_[id_][info_commune] for id_ in liste_id_]
            
        valeurs_non_nulles = list(set(valeurs) - {None})
        assert len(valeurs_non_nulles) <= 1, ('{} {} {} {}'.format(cid, info_commune, set(valeurs)))
        infos_communes_texte[info_commune] = valeurs_non_nulles[0] if len(valeurs_non_nulles) >= 1 else None

    return infos_communes_texte

In [63]:
liste_erreurs = []
infos_communes_par_cid = {}
for base_origine, categorie, cid in liste_texte:
    liste_id_, valeurs_struct_par_id_, valeurs_version_par_id_ = parse_cid(cid)
    try:
        infos_communes_par_cid[cid] = verifie_infos_communes(cid, liste_id_, valeurs_struct_par_id_, valeurs_version_par_id_)
    except AssertionError as e:
        print(e)
        liste_erreurs.append(e)

KeyboardInterrupt: 

Report du 1er passage (64 erreurs) :
```
TNC_non_vigueur_JORF JORFTEXT000000644046 DATE_PUBLI {'2006-09-03', '2006-09-06'}
TNC_en_vigueur_JORF JORFTEXT000000841628 DATE_TEXTE {'1991-08-21', '1991-07-20'}
TNC_en_vigueur_JORF JORFTEXT000000763720 DATE_PUBLI {'2001-12-21', '2001-12-25'}
TNC_en_vigueur_JORF JORFTEXT000000392850 DATE_TEXTE {'1998-03-16', '1998-12-22'}
TNC_en_vigueur_JORF JORFTEXT000000844102 DATE_TEXTE {'1987-06-10', '1987-06-01'}
TNC_en_vigueur_JORF JORFTEXT000000225685 DATE_PUBLI {'2001-11-09', '2001-11-08'}
TNC_en_vigueur_JORF JORFTEXT000000174965 DATE_PUBLI {'1992-07-06', '1992-07-03'}
TNC_non_vigueur_JORF JORFTEXT000000553295 DATE_PUBLI {'1998-02-15', '1998-02-14'}
TNC_en_vigueur_JORF JORFTEXT000000593204 DATE_PUBLI {'2002-01-06', '2002-01-07'}
TNC_en_vigueur_JORF JORFTEXT000000510005 NATURE {'LOI_ORGANIQUE', 'LOI'}
TNC_en_vigueur_JORF JORFTEXT000000244335 DATE_PUBLI {'2007-04-21', '2007-04-22'}
TNC_en_vigueur_JORF JORFTEXT000000408052 DATE_PUBLI {'2001-11-10', '2001-11-09'}
TNC_non_vigueur_JORF JORFTEXT000018699380 DATE_PUBLI {'2008-04-27', '2008-04-29'}
TNC_en_vigueur_JORF JORFTEXT000020506850 DATE_PUBLI {'2009-04-12', '2009-04-14'}
TNC_en_vigueur_JORF JORFTEXT000000759488 DATE_PUBLI {'1999-04-03', '1999-04-04'}
TNC_en_vigueur_JORF JORFTEXT000000499751 DATE_TEXTE {'1989-12-14', '1978-01-13'}
TNC_non_vigueur_JORF JORFTEXT000000447508 DATE_PUBLI {'2005-08-07', '2005-08-06'}
TNC_non_vigueur_JORF JORFTEXT000000424033 DATE_PUBLI {'2005-05-07', '2005-05-12'}
TNC_en_vigueur_JORF JORFTEXT000000412879 DATE_PUBLI {'2002-05-15', '2002-05-05'}
TNC_en_vigueur_JORF JORFTEXT000020473987 DATE_PUBLI {'2009-04-04', '2009-04-05'}
TNC_en_vigueur_JORF JORFTEXT000020795196 DATE_PUBLI {'1939-02-05', '1939-01-19'}
TNC_non_vigueur_JORF JORFTEXT000000264768 DATE_PUBLI {'2006-04-10', '2006-04-11'}
TNC_en_vigueur_JORF JORFTEXT000000418078 DATE_PUBLI {'2003-02-06', '2003-02-08'}
TNC_en_vigueur_JORF JORFTEXT000000468155 DATE_PUBLI {'1984-08-28', '1984-08-29'}
TNC_en_vigueur_JORF JORFTEXT000000598735 DATE_PUBLI {'2002-11-23', '2002-11-25'}
TNC_en_vigueur_JORF JORFTEXT000000590733 DATE_PUBLI {'2001-10-15', '2001-10-13'}
TNC_en_vigueur_JORF JORFTEXT000000250830 DATE_PUBLI {'2004-08-04', '2004-07-31'}
TNC_en_vigueur_JORF JORFTEXT000000226802 DATE_PUBLI {'2002-02-13', '2002-02-07'}
TNC_non_vigueur_JORF JORFTEXT000019025755 DATE_PUBLI {'2008-06-21', '2008-06-20'}
TNC_non_vigueur_JORF JORFTEXT000000468097 DATE_PUBLI {'1983-06-18', '1983-06-21'}
TNC_en_vigueur_JORF JORFTEXT000000403567 DATE_PUBLI {'2002-03-29', '2002-03-25'}
TNC_en_vigueur_JORF JORFTEXT000000819750 DATE_PUBLI {'2006-08-08', '2006-08-13'}
TNC_en_vigueur_JORF JORFTEXT000000367879 NUM {'96-64', '94-1098'}
TNC_en_vigueur_JORF JORFTEXT000018838055 DATE_PUBLI {'2008-05-29', '2008-06-02'}
TNC_non_vigueur_JORF JORFTEXT000000355142 DATE_PUBLI {'1991-06-14', '1991-06-10'}
TNC_non_vigueur_JORF JORFTEXT000000265798 DATE_PUBLI {'2006-01-27', '2006-01-28'}
TNC_en_vigueur_JORF JORFTEXT000018838015 DATE_PUBLI {'2008-05-24', '2008-05-28'}
TNC_en_vigueur_JORF JORFTEXT000019375864 DATE_PUBLI {'2008-09-11', '2008-09-10'}
TNC_non_vigueur_JORF JORFTEXT000000542086 DATE_PUBLI {'1992-08-07', '1992-08-08'}
TNC_non_vigueur_JORF JORFTEXT000000811401 DATE_PUBLI {'2006-01-01', '2005-12-31'}
TNC_non_vigueur_JORF JORFTEXT000018700167 DATE_PUBLI {'2008-05-03', '2008-05-02'}
TNC_non_vigueur_JORF JORFTEXT000020568544 DATE_PUBLI {'2009-05-13', '2009-05-03'}
TNC_en_vigueur_JORF JORFTEXT000000777708 DATE_PUBLI {'2003-01-14', '2003-01-11'}
TNC_en_vigueur_JORF JORFTEXT000000213264 DATE_PUBLI {'1999-12-11', '1999-12-10'}
TNC_en_vigueur_JORF JORFTEXT000000233691 DATE_PUBLI {'2004-05-10', '2004-05-11'}
TNC_en_vigueur_JORF JORFTEXT000000327186 DATE_TEXTE {'1991-10-31', '1991-12-18'}
TNC_en_vigueur_JORF JORFTEXT000020128976 DATE_PUBLI {'2009-01-22', '2009-01-23'}
TNC_en_vigueur_JORF JORFTEXT000000755111 DATE_PUBLI {'2000-12-23', '2000-12-22'}
TNC_non_vigueur_JORF JORFTEXT000000336642 DATE_PUBLI {'1972-01-16', '1972-01-14'}
TNC_en_vigueur_JORF JORFTEXT000000821801 DATE_PUBLI {'2006-12-30', '2007-01-01'}
TNC_non_vigueur_JORF JORFTEXT000000321578 DATE_TEXTE {'1987-03-19', '1980-04-28'}
TNC_en_vigueur_JORF JORFTEXT000019917301 DATE_PUBLI {'2008-12-17', '2008-12-14'}
TNC_non_vigueur_JORF JORFTEXT000000591402 DATE_PUBLI {'2001-10-21', '2001-10-18'}
TNC_en_vigueur_JORF JORFTEXT000000332963 NUM {'87-801', '87-802'}
TNC_non_vigueur_JORF JORFTEXT000000811398 DATE_PUBLI {'2006-01-01', '2005-12-31'}
TNC_en_vigueur_JORF JORFTEXT000000256844 DATE_PUBLI {'2004-10-15', '2004-10-16'}
TNC_en_vigueur_JORF JORFTEXT000000415961 DATE_PUBLI {'2002-09-25', '2002-09-24'}
TNC_non_vigueur_JORF JORFTEXT000019341856 DATE_PUBLI {'2008-09-05', '2008-09-06'}
TNC_non_vigueur_JORF JORFTEXT000000391926 DATE_PUBLI {'2001-03-12', '2001-03-15'}
TNC_en_vigueur_JORF JORFTEXT000000244450 DATE_PUBLI {'2003-11-12', '2003-11-15'}
TNC_en_vigueur_JORF JORFTEXT000000247419 DATE_PUBLI {'2004-02-22', '2004-02-26'}
TNC_en_vigueur_JORF JORFTEXT000000465978 DATE_PUBLI {'2007-03-13', '2007-03-14'}
TNC_en_vigueur_JORF JORFTEXT000000772680 DATE_PUBLI {'2001-09-19', '2001-09-18'}
TNC_en_vigueur_JORF JORFTEXT000000316622 DATE_TEXTE {'1991-04-12', '1991-03-29'}
```

In [13]:
len(liste_erreurs)

0

In [14]:
liste_cid_anomalie = [str(erreur).split(' ')[1] for erreur in liste_erreurs]
', '.join(liste_cid_anomalie)

''

Ajout de la colonne `champs_communs_coherents` :
```
alter table texte add champs_communs_coherents boolean;
```

In [ ]:
connection = psycopg2.connect(dbname='loi', user='loi', password='baba')
curseur = connection.cursor()

for cid, categorie in liste_texte:
    curseur.execute("select etat from texte where cid = %s;", (cid,))
    resultats_requete = curseur.fetchall()
    assert len(resultats_requete) == 1
    etat_vieux = resultats_requete[0][0]
    
    champs_communs_coherents = (cid not in liste_cid_anomalie)
    
    etat = 'valide' if ((etat_vieux == 'valide') and champs_communs_coherents) else 'anomalie'
    
    curseur.execute("update texte set champs_communs_coherents = %s, etat = %s where cid = %s;",
                    (champs_communs_coherents, etat, cid))
    connection.commit()
    
curseur.close()
connection.close()

95599 textes valides

## Vérifie les informations particulières à une version

In [15]:
def verifie_infos_particulieres(categorie, cid, liste_id_, valeurs_struct_par_id_, valeurs_version_par_id_):
    for info_particuliere in infos_particulieres:
        for id_ in liste_id_:
            assert id_ == valeurs_version_par_id_[id_]['ID']
            
            valeur_struct = valeurs_struct_par_id_[id_][info_particuliere]
            valeur_version = valeurs_version_par_id_[id_][info_particuliere]
            
            assert valeur_struct == valeur_version, ('{} {} {} {} != {}'.format(
                categorie, cid, info_particuliere, valeur_struct, valeur_version))

In [16]:
liste_erreurs = []
for cid, categorie in liste_texte:
    liste_id_, valeurs_struct_par_id_, valeurs_version_par_id_ = parse_cid(categorie, cid)
    try:
        verifie_infos_particulieres(categorie, cid, liste_id_, valeurs_struct_par_id_, valeurs_version_par_id_)
    except AssertionError as e:
        print(e)
        liste_erreurs.append(e)

In [17]:
len(liste_erreurs)

0

## Unicité des id\_ (un même id\_ n'apparaît que pour un seul cid)

In [18]:
liste_id_par_cid = {}
for cid, categorie in liste_texte:
    enfants_fichier, enfants_dossier = trouve_enfants(os.path.join(racine_source, categorie, cid, 'struct'))
    liste_id_ = [enfant_fichier[:20] for enfant_fichier in enfants_fichier]

    liste_id_par_cid[cid] = liste_id_

In [33]:
Counter([len(l) for l in liste_id_par_cid.values()])  # nombre de textes ayant n versions

Counter({1: 93612,
         2: 1816,
         3: 127,
         4: 12,
         5: 8,
         6: 8,
         7: 4,
         8: 4,
         9: 4,
         10: 1,
         13: 1})

In [19]:
liste_id_ = [id_ for cid, l in liste_id_par_cid.items() for id_ in l]
set_id_ = set(liste_id_)

In [20]:
len(liste_id_), len(set_id_)  # 1er passage : (97882, 97881)

(97880, 97880)

In [21]:
Counter(Counter(liste_id_).values())  # 1er passage : Counter({1: 97880, 2: 1})

Counter({1: 97880})

In [22]:
dict_id_duplication = {id_: nb for id_, nb in Counter(liste_id_).items() if nb > 1}

In [23]:
dict_id_duplication  # 1er passage : {'LEGITEXT000006077178': 2}

{}

In [24]:
liste_id_duplication = list(dict_id_duplication.keys())
ensemble_id_duplication = set(liste_id_duplication)
liste_id_duplication  # 1er passage : ['LEGITEXT000006077178']

[]

In [25]:
liste_cid_duplication = [cid for cid, l in liste_id_par_cid.items() if ensemble_id_duplication & set(l)]

In [26]:
liste_cid_duplication  # 1er passage : ['JORFTEXT000000343573', 'JORFTEXT000000160046']

[]

Ajout de la colonne `non_duplication_id_` :
```
alter table texte add non_duplication_id_ boolean;
```

In [ ]:
connection = psycopg2.connect(dbname='loi', user='loi', password='baba')
curseur = connection.cursor()

for cid, categorie in liste_texte:
    curseur.execute("select etat from texte where cid = %s;", (cid,))
    resultats_requete = curseur.fetchall()
    assert len(resultats_requete) == 1
    etat_vieux = resultats_requete[0][0]
    
    non_duplication_id_ = (cid not in liste_cid_duplication)
    
    etat = 'valide' if ((etat_vieux == 'valide') and non_duplication_id_) else 'anomalie'
    
    curseur.execute("update texte set non_duplication_id_ = %s, etat = %s where cid = %s;",
                    (non_duplication_id_, etat, cid))
    connection.commit()
    
curseur.close()
connection.close()

95597 textes valides

## Regroupe les informations particulières à une version

In [27]:
def regroupe_infos_particulieres(categorie, cid, id_, valeurs_struct_par_id_, valeurs_version_par_id_):
    info_particuliere_version = {
        'id_': id_,
        'cid': cid,
        'categorie': categorie,
        'ANCIEN_ID': valeurs_version_par_id_[id_]['ANCIEN_ID'],
        'TITRE': valeurs_version_par_id_[id_]['TITRE'],
        'TITREFULL': valeurs_version_par_id_[id_]['TITREFULL'],
        'ETAT': valeurs_version_par_id_[id_]['ETAT'],
        'DATE_DEBUT': valeurs_version_par_id_[id_]['DATE_DEBUT'],
        'DATE_FIN': valeurs_version_par_id_[id_]['DATE_FIN'],
        'VISAS': valeurs_version_par_id_[id_]['VISAS'],
        'SIGNATAIRES': valeurs_version_par_id_[id_]['SIGNATAIRES'],
        'TP': valeurs_version_par_id_[id_]['TP'],
        'NOTA': valeurs_version_par_id_[id_]['NOTA'],
        'ABRO': valeurs_version_par_id_[id_]['ABRO'],
        'RECT': valeurs_version_par_id_[id_]['RECT'],
    }
    
    return info_particuliere_version

In [28]:
infos_particulieres_par_id_ = {}
for cid, categorie in liste_texte:
    liste_id_, valeurs_struct_par_id_, valeurs_version_par_id_ = parse_cid(categorie, cid)
    
    for id_ in liste_id_:
        assert id_ not in infos_particulieres_par_id_, (categorie, cid, id_)
        infos_particulieres_par_id_[id_] = regroupe_infos_particulieres(categorie, cid, id_, valeurs_struct_par_id_, valeurs_version_par_id_)


## Vérification de la liste des versions

In [ ]:
VERSIONS

In [34]:
cid, categorie = liste_texte[0]

In [35]:
infos_communes_par_cid[cid]['VERSIONS']

frozenset({FrozenDict({'fin': '2999-01-01', 'id_': 'JORFTEXT000031393629', 'num': '2015-1364', 'debut': '2999-01-01', 'etat': ''}),
           FrozenDict({'fin': '2999-01-01', 'id_': 'LEGITEXT000031394275', 'num': '2015-1364', 'debut': '2015-11-01', 'etat': 'VIGUEUR_DIFF'})})

In [36]:
liste_id_par_cid[cid]

['LEGITEXT000031394275']

In [37]:
cid

'JORFTEXT000031393629'

In [39]:
cid, categorie = liste_texte[2]

In [47]:
list(zip([len(infos_communes_par_cid[cid]['VERSIONS']) for cid, categorie in liste_texte], 
         [len(liste_id_par_cid[cid]) for cid, categorie in liste_texte],
         liste_texte))

[(2, 1, ('JORFTEXT000031393629', 'TNC_non_vigueur_JORF')),
 (2, 1, ('JORFTEXT000000559911', 'TNC_en_vigueur_JORF')),
 (2, 1, ('JORFTEXT000024725323', 'TNC_en_vigueur_JORF')),
 (2, 1, ('JORFTEXT000000559950', 'TNC_non_vigueur_JORF')),
 (2, 1, ('JORFTEXT000026844221', 'TNC_non_vigueur_JORF')),
 (2, 1, ('JORFTEXT000000378678', 'TNC_non_vigueur_JORF')),
 (2, 1, ('JORFTEXT000029503799', 'TNC_en_vigueur_JORF')),
 (1, 1, ('LEGITEXT000006074014', 'TNC_non_vigueur_LEGI')),
 (2, 1, ('JORFTEXT000024940441', 'TNC_en_vigueur_JORF')),
 (2, 1, ('JORFTEXT000000747077', 'TNC_en_vigueur_JORF')),
 (2, 1, ('JORFTEXT000024357913', 'TNC_en_vigueur_JORF')),
 (2, 1, ('JORFTEXT000023724216', 'TNC_en_vigueur_JORF')),
 (2, 1, ('JORFTEXT000020040783', 'TNC_en_vigueur_JORF')),
 (2, 1, ('JORFTEXT000000695277', 'TNC_non_vigueur_JORF')),
 (2, 1, ('JORFTEXT000000387769', 'TNC_non_vigueur_JORF')),
 (2, 1, ('JORFTEXT000000728204', 'TNC_en_vigueur_JORF')),
 (2, 1, ('JORFTEXT000000215844', 'TNC_non_vigueur_JORF')),
 (2, 1

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


## Mise en base des informations communes à un cid

Ajout des nouvelles colonnes :
```
alter table texte
    add ID_ELI text,
    add ID_ELI_ALIAS text,
    add NATURE text,
    add NUM text,
    add NUM_SEQUENCE text,
    add NOR text,
    add DATE_PUBLI text,
    add DATE_TEXTE text,
    add DERNIERE_MODIFICATION text,
    add ORIGINE_PUBLI text,
    add PAGE_DEB_PUBLI text,
    add PAGE_FIN_PUBLI text,
    add AUTORITE text,
    add MINISTERE text,
;
```


In [ ]:
connection = psycopg2.connect(dbname='loi', user='loi', password='baba')
curseur = connection.cursor()

for cid, categorie in liste_texte:
    infos = infos_communes_par_cid[cid]
    valeurs = {
        'ID_ELI': infos_communes_par_cid['ID_ELI'],
        'ID_ELI_ALIAS': infos_communes_par_cid['ID_ELI_ALIAS'],
        'NATURE': infos_communes_par_cid['NATURE'],
        'NUM': infos_communes_par_cid['NUM'],
        'NUM_SEQUENCE': infos_communes_par_cid['NUM_SEQUENCE'],
        'NOR': infos_communes_par_cid['NOR'],
        'DATE_PUBLI': infos_communes_par_cid['DATE_PUBLI'],
        'DATE_TEXTE': infos_communes_par_cid['DATE_TEXTE'],
        'DERNIERE_MODIFICATION': infos_communes_par_cid['DERNIERE_MODIFICATION'],
        'ORIGINE_PUBLI': infos_communes_par_cid['ORIGINE_PUBLI'],
        'PAGE_DEB_PUBLI': infos_communes_par_cid['PAGE_DEB_PUBLI'],
        'PAGE_FIN_PUBLI': infos_communes_par_cid['PAGE_FIN_PUBLI'],
        'AUTORITE': infos_communes_par_cid['AUTORITE'],
        'MINISTERE': infos_communes_par_cid['MINISTERE'],
        'cid': cid,
    }

    curseur.execute("""
        update article set
            ID_ELI = %(ID_ELI)s,
            ID_ELI_ALIAS = %(ID_ELI_ALIAS)s,
            NATURE = %(NATURE)s,
            NUM = %(NUM)s,
            NUM_SEQUENCE = %(NUM_SEQUENCE)s,
            NOR = %(NOR)s,
            DATE_PUBLI = %(DATE_PUBLI)s,
            DATE_TEXTE = %(DATE_TEXTE)s,
            DERNIERE_MODIFICATION = %(DERNIERE_MODIFICATION)s,
            ORIGINE_PUBLI = %(ORIGINE_PUBLI)s,
            PAGE_DEB_PUBLI = %(PAGE_DEB_PUBLI)s,
            PAGE_FIN_PUBLI = %(PAGE_FIN_PUBLI)s,
            AUTORITE = %(AUTORITE)s,
            MINISTERE = %(MINISTERE)s,
            where cid = %(cid)s;
        """, valeurs)
    connection.commit()
    
curseur.close()
connection.close()

## Mise en base des informations particulières à une version

In [ ]:
VERSIONS

## Mise en base des liens entre une version et une autre version

In [ ]:
LIENS

In [7]:
curseur.close()
connection.close()

## Mise en base des liens entre un texte et un article

In [ ]:
LIENS_ART

## Mise en base des liens entre un texte et une section_ta

In [ ]:
LIENS_SECTION_TA

## Mise en base des versions futures d'un texte

In [ ]:
VERSIONS_A_VENIR

In [ ]:
[infos_communes_par_cid[cid]['VERSIONS_A_VENIR'] for cid, categorie in liste_texte]